In [ ]:
!pip install pandas sqlalchemy

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os

# Create a SQLite database file
db_file = 'sales_data.db'
if os.path.exists(db_file):
    os.remove(db_file)
engine = create_engine(f'sqlite:///{db_file}')

# --- Simulate 5k+ Sales Transactions ---
np.random.seed(42)
num_transactions = 5500

# Generate a date range
dates = pd.date_range(start='2023-01-01', end='2024-01-31', periods=num_transactions).to_list()

# Generate different product categories and regions
products = ['Electronics', 'Home Goods', 'Apparel', 'Books', 'Groceries']
regions = ['North America', 'Europe', 'Asia', 'South America']

# Create a DataFrame
data = {
    'transaction_id': range(1, num_transactions + 1),
    'transaction_date': dates,
    'product_category': np.random.choice(products, num_transactions),
    'region': np.random.choice(regions, num_transactions, p=[0.4, 0.3, 0.2, 0.1]),
    'sale_amount': np.random.uniform(10.0, 500.0, num_transactions).round(2),
    'quantity': np.random.randint(1, 6, num_transactions)
}
df_raw = pd.DataFrame(data)

# Introduce some "dirty" data to simulate real-world challenges
df_raw.loc[100, 'sale_amount'] = -250.0  # Negative value
df_raw.loc[200, 'region'] = None        # Missing value
df_raw.loc[300, 'product_category'] = 'electronics' # Inconsistent casing

# Save the raw data to the database
df_raw.to_sql('raw_sales', con=engine, if_exists='replace', index=False)
print(f"Successfully created 'raw_sales' table with {len(df_raw)} records in {db_file}")
print("Sample of raw data:")
print(pd.read_sql_query("SELECT * FROM raw_sales LIMIT 5;", con=engine))

In [ ]:
SELECT
    -- Clean 'product_category' by capitalizing the first letter
    UPPER(SUBSTR(product_category, 1, 1)) || LOWER(SUBSTR(product_category, 2)) AS cleaned_product_category,
    
    -- Filter out nulls and convert to uppercase for consistency
    CASE
        WHEN region IS NULL THEN 'Unknown'
        ELSE UPPER(region)
    END AS cleaned_region,
    
    -- Aggregate by date (monthly)
    STRFTIME('%Y-%m', transaction_date) AS month,
    
    -- Calculate aggregated metrics
    COUNT(transaction_id) AS total_transactions,
    SUM(CASE WHEN sale_amount > 0 THEN sale_amount ELSE 0 END) AS total_sales, -- Clean negative sales
    AVG(CASE WHEN sale_amount > 0 THEN sale_amount ELSE 0 END) AS avg_sale_amount,
    SUM(quantity) AS total_quantity_sold
FROM
    raw_sales
WHERE
    -- Filter out transactions with no region or invalid sale amounts
    region IS NOT NULL AND sale_amount > 0
GROUP BY
    month,
    cleaned_region,
    cleaned_product_category
ORDER BY
    month,
    total_sales DESC;

In [ ]:
SELECT
    STRFTIME('%Y-%m', transaction_date) AS month,
    region,
    SUM(sale_amount) AS monthly_sales
FROM
    raw_sales
WHERE
    region IS NOT NULL AND sale_amount > 0
GROUP BY
    month,
    region
ORDER BY
    month,
    region;

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

db_file = 'sales_data.db'
engine = create_engine(f'sqlite:///{db_file}')

# Define the cleaned and aggregated query
cleaned_sales_query = """
SELECT
    UPPER(SUBSTR(product_category, 1, 1)) || LOWER(SUBSTR(product_category, 2)) AS product_category,
    CASE WHEN region IS NULL THEN 'Unknown' ELSE region END AS region,
    STRFTIME('%Y-%m-%d', transaction_date) AS transaction_date,
    SUM(sale_amount) AS total_sales
FROM
    raw_sales
WHERE
    sale_amount > 0 AND region IS NOT NULL
GROUP BY
    transaction_date,
    region,
    product_category
ORDER BY
    transaction_date;
"""

# Execute the query and save to a new CSV file
df_clean = pd.read_sql_query(cleaned_sales_query, con=engine)
df_clean.to_csv('cleaned_sales_data.csv', index=False)
print("Cleaned data saved to 'cleaned_sales_data.csv' for Tableau.")